<a href="https://colab.research.google.com/github/unihwan/punctuation_restoration/blob/main/assignment_2_word_level.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#for google drive directory (colab)
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Change your path location here 

In [ ]:
filepath = ""

In [ ]:
#Importing library 
import numpy as np
from keras.models import Model
from keras.layers import Input, LSTM, Dense,Embedding,Bidirectional,Activation, dot, concatenate,TimeDistributed, GRU
from keras.preprocessing.text import Tokenizer
from keras.initializers import *
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
import time , random
import os
import math
import re
import pickle
from nltk.tokenize import word_tokenize, sent_tokenize

In [ ]:
# function to get tokenizer
def tokenize(lang):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='', lower=False, num_words=None)
    tokenizer.fit_on_texts(lang)
    return tokenizer

In [ ]:
#Global value 
NUM_SENTENCES = 600000
MAX_NUM_WORDS = 4000

In [ ]:
with open(filepath + 'norm.txt', 'r',encoding='cp1252') as f:
    testinput = f.read().split('\n')
print(testinput)

['berkoban adalah satu bentuk ujian daripada allah s.w.t', 'kerana dalam menjalani kehidupan di dunia ini kita harus melakukan pengorbanan', 'hidup tanpa pengorbanan adalah kehidupan yang gagal', 'dalam menjalani hidup yang baik kita mesti berkorban masa dan tenaga misalnya berusaha mencari rezeki yang halal bagi mengelakkan kemiskinan', 'a merupakan tuntutan hidup kata pensyarah jabatan dakwah dan pembangunan insan akademi pengajian islam universiti malaya roslan mohamed mengenai konsep berkorban dalam islam', 'korban berasal daripada perkataan arab iaitu udhiyah', 'a menjurus kepada peristiwa pengorbanan satu keluarga iaitu nabi ibrahim dan isterinya hajar yang sanggup mengorbankan sesuatu yang amat mereka sayangi iaitu anaknya nabi ismail demi kasih dan cinta kepada allah s.w.t', 'ini adalah nilai pengorbanan yang cukup tinggi di sisi allah s.w.t', 'kerana ia bukan sahaja menguji kesabaran nabi ibrahim tetapi menduga sejauh mana ketakwaan nabi ibrahim', 'maka allah s.w.t menggantika

In [ ]:
with open(filepath + 'unnorm.txt', 'r',encoding='cp1252') as f:

    testoutput = f.read().split('\n')

print(testoutput)

['"Berkoban adalah satu bentuk ujian daripada Allah s.w.t. ', 'kerana dalam menjalani kehidupan di dunia ini, kita harus melakukan pengorbanan. ', 'Hidup tanpa pengorbanan adalah kehidupan yang gagal.', 'Dalam menjalani hidup yang baik, kita mesti berkorban masa dan tenaga misalnya berusaha mencari rezeki yang halal bagi mengelakkan kemiskinan. ', 'a merupakan tuntutan hidup," kata pensyarah Jabatan Dakwah dan Pembangunan Insan, Akademi Pengajian Islam, Universiti Malaya, Roslan Mohamed mengenai konsep berkorban dalam Islam.', 'Korban berasal daripada perkataan Arab iaitu udhiyah.', 'a menjurus kepada peristiwa pengorbanan satu keluarga iaitu Nabi Ibrahim dan isterinya, Hajar yang sanggup mengorbankan sesuatu yang amat mereka sayangi iaitu anaknya, Nabi Ismail demi kasih dan cinta kepada Allah s.w.t.', '"Ini adalah nilai pengorbanan yang cukup tinggi di sisi Allah s.w.t. ', 'kerana ia bukan sahaja menguji kesabaran Nabi Ibrahim tetapi menduga sejauh mana ketakwaan Nabi Ibrahim. ', 'Mak

In [ ]:
input_sentences = []
output_sentences = []
output_sentences_inputs = []

In [ ]:
with open(filepath + '2008.txt', 'r',encoding='cp1252') as f:

    lines = f.read().split('\n')

In [ ]:
input_texts = []
target_texts = []
input_sentences1 =[]
target_sentences = []

In [ ]:
#prepare corpus
#remove punctuation
#added \t and \n
for line in lines:
     input_text = line.lower()
     input_text = re.sub(r"[^a-zA-Z0-9]+", " ", input_text)
     input_text =re.sub(r'[ ]{2,}',r' ',input_text)
     #target_text =line.strip()
     target_text = '\t '+line +' \n'  
     input_sentences1.append(input_text)
     target_sentences.append(target_text)

In [ ]:
for i in range(len(input_sentences1)):
  if(len(lines[i])<=200):
    target_sentences[i]=re.sub(r'([!"#$%&\'‘“”’()*+,-./:;<=>…?@[\]^_`{|}~\n\t])',r' \1 ' ,target_sentences[i])
    target_sentences[i]=re.sub(r'[ ]{2,}',r' ',target_sentences[i])
    input_sentences.append(input_sentences1[i])
    output = target_sentences[i]

    output_sentence = output 
    output_sentence_input = output

    output_sentences.append(output_sentence)
    output_sentences_inputs.append(output_sentence_input)

In [ ]:
print("num samples input:", len(input_sentences))
print("num samples output:", len(output_sentences))
print("num samples output input:", len(output_sentences_inputs))

num samples input: 460949
num samples output: 460949
num samples output input: 460949


In [ ]:
input_tokenizer = Tokenizer(lower=False,num_words=MAX_NUM_WORDS,oov_token='<oov>')
input_tokenizer.fit_on_texts(input_sentences)
input_integer_seq = input_tokenizer.texts_to_sequences(input_sentences)

word2idx_inputs = input_tokenizer.word_index
print('Total unique words in the input: %s' % len(word2idx_inputs))

max_input_len = max(len(sen) for sen in input_integer_seq)
print("Length of longest sentence in input: %g" % max_input_len)

Total unique words in the input: 83249
Length of longest sentence in input: 44


In [ ]:
output_tokenizer = Tokenizer(lower=False,num_words=MAX_NUM_WORDS, filters='',oov_token='<oov>')
output_tokenizer.fit_on_texts(output_sentences + output_sentences_inputs)
output_integer_seq = output_tokenizer.texts_to_sequences(output_sentences)
output_input_integer_seq = output_tokenizer.texts_to_sequences(output_sentences_inputs)

word2idx_outputs = output_tokenizer.word_index
print('Total unique words in the output: %s' % len(word2idx_outputs))

num_words_output = len(word2idx_outputs) + 1
max_out_len = max(len(sen) for sen in output_integer_seq)
print("Length of longest sentence in the output: %g" % max_out_len)

Total unique words in the output: 104515
Length of longest sentence in the output: 64


In [ ]:
# enc_token = save tokenized input characters
enc_token = tokenize(input_sentences)
reverse_enc_dict = {i: c for c, i in enc_token.word_index.items()}

# dec_token = save tokenized target characters
dec_token = tokenize(output_sentences)
reverse_dec_dict = {i: c for c, i in dec_token.word_index.items()}

a = {'max_encoder_seq_length': max_input_len,
     'max_decoder_seq_length': max_out_len,
     'enc_vocab_size': MAX_NUM_WORDS+1,
     'dec_vocab_size': MAX_NUM_WORDS+1,
     'reverse_enc_dict': reverse_enc_dict,
     'reverse_dec_dict': reverse_dec_dict,
     'enc_token': enc_token,
     'dec_token': dec_token}

with open(filepath + 'params_word_1.pkl', 'wb') as handle:
   pickle.dump(a, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
def get_text_encodings_generate(texts):
    result = []
    for x in texts[0].split(' '):
      try:
        if enc_token.word_index[x] > MAX_NUM_WORDS:
          result.append(enc_token.word_index['<oov>'])
        else:  
          result.append(enc_token.word_index[x])
      except:
        continue
    pad_seq = pad_sequences([result], maxlen=max_input_len,
                            padding='post')
    pad_seq = to_categorical(pad_seq, num_classes=MAX_NUM_WORDS+1)
    return pad_seq

In [ ]:
def get_text_decodings_generate(texts):
    result = []
    for x in texts[0].split(' '):
      try:
        if dec_token.word_index[x] > MAX_NUM_WORDS:
          result.append(dec_token.word_index['<oov>'])
        else:  
          result.append(dec_token.word_index[x])
      except:
        continue
    pad_seq = pad_sequences([result], maxlen=max_out_len,
                            padding='post')
    pad_seq = to_categorical(pad_seq, num_classes=MAX_NUM_WORDS+1)
  
    return pad_seq

In [ ]:
def generate_batch(X, y, batch_size=32):
    ''' Generate a batch of data '''
    while True:
        for j in range(0, len(X), batch_size):
            
            encoder_input_data = np.zeros((batch_size, max_input_len,MAX_NUM_WORDS+1), dtype='float32')

            decoder_input_data = np.zeros((batch_size, max_out_len, MAX_NUM_WORDS+1), dtype='float32')

            decoder_target_data = np.zeros((batch_size, max_out_len, MAX_NUM_WORDS+1), dtype='float32')


            for i, (input_sentence, target_sentence) in enumerate(zip(X[j:j+batch_size], y[j:j+batch_size])):

                encoder_input_data[i] = get_text_encodings_generate([input_sentence])
                decoder_input_data[i] = get_text_decodings_generate([target_sentence])
                decoder_target_data[i] = get_text_decodings_generate([target_sentence[2 :]])

            yield([encoder_input_data, decoder_input_data], decoder_target_data)

In [ ]:
encoder_inputs = Input(shape=(None,MAX_NUM_WORDS+1,))
encoder = Bidirectional(LSTM(128,return_sequences=True, return_state=True),merge_mode='concat')
encoder_outputs, forward_h, forward_c, backward_h, backward_c = encoder(encoder_inputs)

encoder_h = concatenate([forward_h, backward_h])
encoder_c = concatenate([forward_c, backward_c])

decoder_inputs = Input(shape=(None, MAX_NUM_WORDS+1,))
decoder_lstm = LSTM(256, return_sequences=True)
decoder_outputs = decoder_lstm(decoder_inputs, initial_state=[encoder_h, encoder_c])

attention = dot([decoder_outputs, encoder_outputs], axes=(2, 2))
attention = Activation('softmax', name='attention')(attention)
context = dot([attention, encoder_outputs], axes=(2, 1))
decoder_combined_context = concatenate([context, decoder_outputs])

output = TimeDistributed(Dense(128, activation="relu"))(decoder_combined_context)
output = TimeDistributed(Dense(MAX_NUM_WORDS+1, activation="softmax"))(output)

model = Model([encoder_inputs, decoder_inputs], [output])

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, 4001)] 0                                            
__________________________________________________________________________________________________
bidirectional (Bidirectional)   [(None, None, 256),  4229120     input_1[0][0]                    
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None, 4001)] 0                                            
__________________________________________________________________________________________________
concatenate (Concatenate)       (None, 256)          0           bidirectional[0][1]              
                                                                 bidirectional[0][3]          

In [ ]:
print('input_texts length :' , len(input_sentences))

input_texts length : 460949


In [ ]:
def train_model(batch_size = 256, epochs=5):
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit_generator(
        generator=generate_batch(X=input_sentences, y=output_sentences, batch_size=batch_size),
        steps_per_epoch=math.ceil(len(input_sentences)/batch_size),
        epochs=epochs,
        verbose=1,
        validation_data = generate_batch(X=testinput, y=testoutput, batch_size=batch_size),
        validation_steps=math.ceil(len(testinput)/batch_size),
        workers=1,
    )

In [ ]:
train_model(epochs=1)

weights_path = os.path.join(filepath, 'checkpoint_word_1.h5')
model.save_weights(weights_path)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


 244/1801 [===>..........................] - ETA: 22:58 - loss: 3.1276 - accuracy: 0.7212

KeyboardInterrupt: ignored

In [ ]:
weights_path = os.path.join(filepath, 'checkpoint_word_1.h5')
model.load_weights(weights_path)

In [ ]:
# Load parameters
with open(filepath + 'params_word_1.pkl', "rb") as file:
    parameters = pickle.load(file)

In [ ]:
# TESTING
def get_text_encodings(texts):
    result = []
    for x in texts[0].split(' '):
      try:
        if parameters["enc_token"].word_index[x] > MAX_NUM_WORDS:
          result.append(parameters["enc_token"].word_index['<oov>'])
        else:  
          result.append(parameters["enc_token"].word_index[x])
      except:
        continue
    pad_seq = pad_sequences([result], maxlen=parameters["max_encoder_seq_length"],
                            padding='post')
    pad_seq = to_categorical(pad_seq, num_classes=parameters["enc_vocab_size"])
    return pad_seq

In [ ]:
def get_extra_chars():
    allowed_extras = []
    for d_c, d_i in parameters["dec_token"].word_index.items():
        
        if d_c.lower() not in parameters["enc_token"].word_index:
            allowed_extras.append(d_i)
    return allowed_extras

In [ ]:
def decode(input_text, allowed_extras, batch_size=128):
    input_texts = [input_text]
    input_texts_c = input_texts.copy()
    out_dict = {}
    input_sequences = get_text_encodings(input_texts)
    outputs = [""] * len(input_sequences+1)
    target_text = "\t"
    parameters["reverse_dec_dict"][0] = "\n"

    target_seq = parameters["dec_token"].texts_to_sequences([target_text] * len(input_sequences))
    target_seq = pad_sequences(target_seq, maxlen=parameters["max_decoder_seq_length"],
                               padding="post")

    target_seq_hot = to_categorical(target_seq, num_classes=parameters["dec_vocab_size"])

    extra_char_count = [0] * len(input_texts)
    prev_char_index = [0] * len(input_texts)
    i = 0
    
    while len(input_texts) != 0:

        curr_char_index = [i - extra_char_count[j] for j in range(len(input_texts))]
        input_encodings = np.argmax(input_sequences, axis=2)
        cur_inp_list = [input_encodings[_][curr_char_index[_]] if curr_char_index[_] < len(input_texts[_]) else 0 for _
                        in range(len(input_texts))]
        output_tokens = model.predict([input_sequences, target_seq_hot], batch_size=batch_size)

        sampled_possible_indices = np.argsort(output_tokens[:, i, :])[:, ::-1].tolist()
        sampled_token_indices = []
        for j, per_char_list in enumerate(sampled_possible_indices):
            
            for index in per_char_list:
                if index in allowed_extras:
                    if parameters["reverse_dec_dict"][index] != '\n' and cur_inp_list[j] != 0:
                        continue
                    elif parameters["reverse_dec_dict"][index] != '\n' and prev_char_index[j] in allowed_extras:
                        continue
                    sampled_token_indices.append(index)
                    extra_char_count[j] += 1
                    break
                elif parameters["reverse_dec_dict"][index] != '\n' and parameters["enc_token"].word_index[parameters["reverse_dec_dict"][index].lower()] == cur_inp_list[j]:
                    sampled_token_indices.append(index)
                    break

        sampled_chars = []
        for index in sampled_token_indices:
            sampled_chars.append(parameters["reverse_dec_dict"][index])
        outputs = [outputs[j] +" " +sampled_chars[j] for j, output in enumerate(outputs)]
        end_indices = sorted([index for index, char in enumerate(sampled_chars) if char == '\n'], reverse=True)
        for index in end_indices:
            out_dict[input_texts[index]] = outputs[index].strip()
            del outputs[index]
            del input_texts[index]
            del extra_char_count[index]
            del sampled_token_indices[index]
            input_sequences = np.delete(input_sequences, index, axis=0)
            target_seq = np.delete(target_seq, index, axis=0)
        if i == parameters["max_decoder_seq_length"] - 1 or len(input_texts) == 0:
            break
        target_seq[:, i + 1] = sampled_token_indices
        target_seq_hot = to_categorical(target_seq, num_classes=parameters["dec_vocab_size"])
        prev_char_index = sampled_token_indices
        i += 1
    outputs = [out_dict[text] for text in input_texts_c]
    return outputs[0]

In [ ]:
extra = get_extra_chars()

In [ ]:
decode("dalam kejadian itu muhammad israf izzuddin sembilan belas yang menunggang motosikal dirempuh kereta dipandu lelaki itu yang berlaku pada pukul dua belas tengah malam", extra)

'Dalam kejadian itu Muhammad Sembilan yang motosikal kereta lelaki itu yang berlaku pada pukul dua tengah malam .'

In [ ]:
with open(filepath + 'testset.txt', 'r',encoding='cp1252') as f:
  test_data = f.read().split('\n')

for line in test_data:
    print("Original Text:" + line)
    line = line.lower() 
    line = re.sub(r"[^a-zA-Z0-9]", " ", line)
    line=re.sub(r'[ ]{2,}',r' ',line) 
    print("Normalized Text:" + line)
    print("Predicted Text:" + decode(line, extra))
    print()


Original Text:Menteri Kanan (Keselamatan), Datuk Seri Ismail Sabri Yaakob, berkata angka itu menjadikan jumlah keseluruhan pekerja asing disahkan dijangkiti COVID-19 seramai 4,096 orang sejak ujian saringan dijalankan pada 1 Disember tahun lalu.
Normalized Text:menteri kanan keselamatan datuk seri ismail sabri yaakob berkata angka itu menjadikan jumlah keseluruhan pekerja asing disahkan dijangkiti covid 19 seramai 4 096 orang sejak ujian saringan dijalankan pada 1 disember tahun lalu 
Predicted Text:Menteri Kanan Keselamatan Datuk Seri Ismail Sabri Yaakob berkata

Original Text:Beliau berkata sebanyak 854 klinik menyertai program saringan terbabit.
Normalized Text:beliau berkata sebanyak 854 klinik menyertai program saringan terbabit 
Predicted Text:Beliau berkata

Original Text:Daripada jumlah ini, sebanyak 4,096 disahkan positif COVID-19 manakala 201,220 lagi adalah negatif.
Normalized Text:daripada jumlah ini sebanyak 4 096 disahkan positif covid 19 manakala 201 220 lagi adalah nega